### Process Tiff Stack

In [1]:
import os
import time
from pvd_par import PVD
from IPython.display import clear_output

### Get data paths

In [2]:
# Function to get files and paths
def scan_directories(data_directory, min_size_bytes):
    dataset_list = []
    session_list = []
    file_list = []

    for root, dirs, files in os.walk(data_directory):
        # Check if the current path contains "exclude" or "movement"
        if "exclude" in root.lower() or "movement" in root.lower() or "missing" in root.lower():
            continue  # Skip this directory

        for file in files:
            if file.endswith('.tif'):
                file_path = os.path.join(root, file)

                # Check file size
                if os.path.getsize(file_path) < min_size_bytes:
                    continue  # Skip this file if it's too small

                path_parts = os.path.normpath(root).split(os.sep)
                if len(path_parts) >= 3:
                    dataset = path_parts[-2]
                    session = path_parts[-1]
                    dataset_list.append(dataset)
                    session_list.append(session)
                    file_list.append(file)

    return dataset_list, session_list, file_list

In [3]:
# Get all tiff stacks and their paths
data_dir = 'S:/pvd_data/'
min_file_size = 1e8  # 100 MB
datasets, sessions, files = scan_directories(data_dir, min_file_size)
print(f"Located {len(files)} tiff stacks")

Located 53 tiff stacks


### Process Data

In [4]:
# Execute pipeline
for ii, file in enumerate(files[:1]):
    clear_output(wait=True)

    # Set path
    data_path = 'S:/pvd_data'
    results_path = 'S:/pvd_analysis'
    dataset = datasets[ii]
    session = sessions[ii]
    #tiff_stack_path = f"{data_path}/{dataset}/{session}/{file}"
    output_path = f"{results_path}/{dataset}/{session}/"

    # Process stack
    print(f"Processing tiff stack {ii}/{len(files)}")
    print(f"{datasets[ii]} {sessions[ii]}")
    pvd = PVD(data_path, dataset, session, file)
    pvd.run_pipeline()



Processing tiff stack 0/53
DataSet00 exp240104_00_01_
Starting pipeline.
Data loaded. Shape: (4, 92, 2044, 2042). Time: 1.81 seconds
Data cropped. New shape: (4, 92, 2000, 2000). Time: 3.21 seconds
Data preprocessed. Processed data shape: [(92, 2000, 2000), (92, 2000, 2000), (92, 2000, 2000), (92, 2000, 2000)]. Time: 86.73 seconds
Skeletonizing timepoint 0
Skeletonizing timepoint 1
Skeletonizing timepoint 2
Skeletonizing timepoint 3
Completed skeletonizing timepoint 3
Completed skeletonizing timepoint 0
Completed skeletonizing timepoint 1
Completed skeletonizing timepoint 2
Data skeletonized. Time: 164.79 seconds
Number of tips per timepoint: [214, 186, 136, 98]
Number of knots per timepoint: [475, 452, 311, 246]
Outer segments found. Number of outer segments per timepoint: [130, 109, 78, 52]. Time: 3.95 seconds
Matched 7 segments across all timepoints.
Segments matched. Number of matched segments per timepoint: [7, 7, 7, 7]. Time: 0.40 seconds
Grouped unmatched segments across each ti

In [5]:
# Save relevant numpy arrays
print(f"Saving data to drive...")
pvd.save_results(output_path, save_tiff=False, save_numpy=False, save_plotly=True, save_labeled_tiff=True)

time.sleep(1)

Saving data to drive...
Skeleton visualizations saved to S:/pvd_analysis/DataSet00/exp240104_00_01_//visualizations
Outer segment visualizations saved to S:/pvd_analysis/DataSet00/exp240104_00_01_//visualizations
Labeled data saved as TIFF files in S:/pvd_analysis/DataSet00/exp240104_00_01_//visualizations


In [7]:
## Code to extract core segment indices and add to matched segments lists
# core_segments = []

# for ii in range(len(self.unmatched_segments):
#     core_segments.append(np.where(pvd.unmatched_segments[ii] == 1))

# for ii in range(len(self.unmatched_segments)):
#     x = list(core_segments[ii][1])
#     y = list(core_segments[ii][2])
#     z = list(core_segments[ii][0])

#     self.matched_segments[ii].append = list(zip(z,x,y))

# print(len(test))

In [8]:
# # Set path to original test file
# data_path = 'S:/pvd_data/'
# results_path = 'S:/pvd_analysis'
# dataset = 'DataSet01'
# session = 'exp240202_01_E_'
# file = 'exp240202_01_E.tif'
# tiff_stack_path = f"{data_path}/{dataset}/{session}_/{file}"
# output_path = f"{results_path}/{dataset}/{session}/"

# # Process stack
# print(f"Processing tiff stack {ii}/{len(files)}")
# print(f"{datasets[ii]} {sessions[ii]}")
# pvd = PVD(data_path, dataset, session, file)
# pvd.run_pipeline()

# # Save relevant numpy arrays
# print(f"Saving data to drive...")
# pvd.save_results(output_path, save_tiff=False, save_numpy=False, save_plotly=True)

# time.sleep(1)

In [9]:
import numpy as np

In [10]:
pvd.skeletonized_data[0].shape

(92, 2000, 2000)

In [11]:
pvd.skeleton_idx = np.where(pvd.skeletonized_data[0] == 1)